In [36]:
!pip install mecab-python3
!pip install sqlalchemy
import gensim
import MeCab
import numpy as np
from scipy import spatial
from sqlite3 import connect, Row
import sys
from statistics import mean
import numpy as np

     |████████████████████████████████| 1.2 MB 8.3 MB/s eta 0:00:01


In [13]:
sentence = '夏、海沿いの静かな町。幸せな夏休みを送るはずだった瑞穂の親友が殺された。瑞穂は密室の謎に叔父である弁護士とともに挑む。その犯人とは。'
targets = []

In [14]:
targets.append('夏、海沿静かな町。幸せな送るはずだっ穂の親友が殺された。密室の謎に弁護士とともに挑む。その犯人とは。'
)
targets.append('静かな町。幸せな夏休みれた。瑞穂は密室の謎に叔父である弁ともに挑む。その犯人とは。'
)
targets.append('夏、海沿いの静かな町。幸せな夏休みを送るはずだった。瑞穂は密室の謎に叔父である弁護士とともに挑む。その犯人とは。'
)

In [15]:
print('loading trained model...')
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('model.vec', binary=False)
print('loaded')
mecab = MeCab.Tagger("/usr/local/lib/mecab/dic/mecab-ipadic-neologd")

loading trained model...
loaded


In [42]:
print(mecab)

<Swig Object of type 'MeCab::Tagger *' at 0x1a25bdaa70>


In [18]:
def avg_feature_vector(sentence, model, num_features):
    words = mecab.parse(sentence).replace(' \n', '').split()  # mecabの分かち書きでは最後に改行(\n)が出力されてしまうため、除去
    words = [words[i] for i in range(len(words)) if i % 2 == 0]
    feature_vec = np.zeros((num_features,), dtype="float32")  # 特徴ベクトルの入れ物を初期化
    exceptions = 0
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except:
            exceptions += 1
    if len(words) - exceptions > 0:
        feature_vec = np.divide(feature_vec, len(word) - exceptions)
    return feature_vec

def mecab_similarity(sentence_1, sentence_2):
    num_features = 300
    sentence_1_avg_vector = avg_feature_vector(sentence_1, word2vec_model, num_features)
    sentence_2_avg_vector = avg_feature_vector(sentence_2, word2vec_model, num_features)
    # １からベクトル間の距離を引いてあげることで、コサイン類似度を計算
    return 1 - spatial.distance.cosine(sentence_1_avg_vector, sentence_2_avg_vector)

In [43]:
_similarities = [{"index":i, "value":0} for i in range(len(targets))]
splited_input = sentence.split('。')
for movie_num, target in enumerate(targets):
    _similarities_per_movie = []
    splited_target = target.split('。')
    for i in range(len(splited_input)):
        try:
            _similarities_per_movie.append(mecab_similarity(splited_input[i], splited_target[i]))
        except:
            pass
    _similarities[movie_num]['value'] = mean(_similarities_per_movie)
print(_similarities)

[{'index': 0, 'value': 0.9605730056762696}, {'index': 1, 'value': 0.9224912643432617}, {'index': 2, 'value': 0.9855925083160401}]


In [44]:
from operator import itemgetter
sorted(_similarities, key=itemgetter('value'), reverse=True)

[{'index': 2, 'value': 0.9855925083160401},
 {'index': 0, 'value': 0.9605730056762696},
 {'index': 1, 'value': 0.9224912643432617}]